# Keypoint-MoSeq 3D Demo

This notebook demonstrates how to fit a keypoint-MoSeq model to **3D keypoint data**. It assumes you already have triangulated keypoints (e.g. from Anipose or SLEAP-anipose) so no additional reconstruction is required.


## Install and Setup
Run the following cell to install keypoint-MoSeq and mount your Google Drive (if using Colab).

In [ ]:
!pip install -U keypoint-moseq
import os
from google.colab import drive
drive.mount('/content/drive')


## Project Directory
Specify a directory to store the project configuration and results.

In [ ]:
import keypoint_moseq as kpms

project_dir = '/content/drive/MyDrive/3d_demo_project'
config = lambda: kpms.load_config(project_dir)


### Setup Project
If you already have a config file in `project_dir` just load it. Otherwise run one of the following setup commands. Replace `anipose_file` with the path to one of your 3D keypoint files.


In [ ]:
# Example: setup from anipose
# anipose_file = '/path/to/recording.csv'
# kpms.setup_project(project_dir, anipose_file=anipose_file)

# Example: manual setup
# bodyparts=[...]  # list of keypoint names
# skeleton=[...]   # list of [start, end] pairs
# kpms.setup_project(project_dir, bodyparts=bodyparts, skeleton=skeleton)


Edit the config as needed. In most cases you will want to set `fps`, `use_bodyparts`, `anterior_bodyparts` and `posterior_bodyparts`.

In [ ]:
kpms.update_config(project_dir, fps=30)


## Load 3D keypoint data
Use `kpms.load_keypoints` with the appropriate format. Common choices are `'anipose'` or `'sleap-anipose'`. The path can be a single file, a directory, or a pattern such as `'/path/to/*.h5'`.

In [ ]:
keypoint_path = '/content/drive/MyDrive/3d_keypoints/*.h5'
coordinates, confidences, bodyparts = kpms.load_keypoints(keypoint_path, 'sleap-anipose')

# format for modeling
data, metadata = kpms.format_data(coordinates, confidences, **config())


## Outlier Interpolation
Large keypoint outliers can be removed before fitting the model.

In [ ]:
kpms.update_config(project_dir, outlier_scale_factor=6.0)
for name in coordinates:
    raw = coordinates[name].copy()
    out = kpms.find_medoid_distance_outliers(raw, **config())
    coordinates[name] = kpms.interpolate_keypoints(raw, out['mask'])
    confidences[name] = np.where(out['mask'], 0, confidences[name])


## Calibration
Annotate a few frames to calibrate confidence scores.

In [ ]:
kpms.calibrate_keypoint_confidence(coordinates, confidences, project_dir, **config())


## Fit Model
First fit an ARHMM for initialization and then fit the full model.

In [ ]:
num_ar_iters = 50
model, model_name = kpms.fit_model(None, data, metadata, project_dir, ar_only=True, num_iters=num_ar_iters)

model, data, metadata, current_iter = kpms.load_checkpoint(project_dir, model_name, iteration=num_ar_iters)
model = kpms.update_hypparams(model, kappa=1e4)
model = kpms.fit_model(model, data, metadata, project_dir, model_name, ar_only=False, start_iter=current_iter, num_iters=current_iter+500)[0]


## Visualization
Plot principal components and trajectories. Grid movies are generated using keypoints only because 3D keypoints cannot be paired with videos directly.

In [ ]:
results = kpms.load_results(project_dir, model_name)
kpms.plot_pcs(coordinates, results, project_dir, model_name, **config())
kpms.generate_trajectory_plots(coordinates, results, project_dir, model_name, **config())
kpms.generate_grid_movies(results, project_dir, model_name, coordinates=coordinates, keypoints_only=True, keypoints_scale=1, use_dims=[0,1], **config())


This completes the basic 3D demo. See the FAQ for more details on working with 3D data.